In [ ]:
!pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.


## Sklearn Models Registration

### session creation

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

### modeling

#### Multiclass classification

In [2]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
from fosforml import register_model
register_model(
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="Mahesh_sklearn_26Aug",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.

Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [ ]:
x_train_df = session.create_dataframe(x_train)

In [ ]:
from fosforml.model_manager import DatasetManager

dataset_manager = DatasetManager(model_name="MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_RANDOMMULTICLASSMODELFORTESTING", version_name="v2", session=session)
dataset_manager.upload_datasets(session=session, datasets={"x_train": x_train_df})

#### Custom Model using score_func

In [ ]:
from snowflake.ml.model import custom_model
import pandas as pd

In [ ]:
class CustomTestModel(custom_model.CustomModel):
    def __init__(self, context: custom_model.CustomModel) -> None:
        super().__init__(context)

    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        score_func = self.context.model_ref('feature_preproc')
        model = self.context.model_ref('model')
        return score_func(
            model,
            input_data
        )

In [ ]:
def feature_preproc(model,input_data):
    return pd.DataFrame(model.predict(input_data),columns=["Predicted"])

mc = custom_model.ModelContext(
    models={
        'model': model,
        'feature_preproc': feature_preproc
    }
)

In [ ]:
custom_test_model = CustomTestModel(mc)

In [ ]:
custom_model_predicted = custom_test_model.predict(x_test)

In [ ]:
custom_model_predicted.head()

In [ ]:
# custom_model_predicted

In [ ]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params

In [ ]:
params

In [ ]:
from snowflake.ml.registry import Registry

In [ ]:
model_reg = Registry(
    session=session,
    database_name="FDC_DEV_VISHWASMAHESHWARI",
    schema_name="FDC_DEV_SCHMEA"
)

In [ ]:
custom_test_model.context.model_refs.keys()

In [ ]:
!python -c "import platform;print(platform.python_version())"

In [ ]:
model_reg.log_model(
       model=custom_test_model,
       model_name="custom_test_model_feature_pre",
       version_name="custom_test_model_v1",
       comment="Testing score function with custom_test_model",
       conda_dependencies=['scikit-learn==1.3.2'],
       metrics={},
       sample_input_data=m_df.drop('Target',axis=1),
       python_version="3.10.13"
)

In [ ]:
from snowflake.ml.modeling.metrics import (confusion_matrix,
                                        accuracy_score,
                                        f1_score, recall_score,
                                        precision_score,
                                        log_loss ,
                                        roc_auc_score ,
                                        roc_curve)

In [ ]:
roc_auc_score(df=self.sf_df,y_true_col_names=self.true_cn, y_score_col_names=self.pred_cn)

In [ ]:
from fosforml import register_model
register_model(
  model_obj=custom_test_model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="Snowflake_dataset",
  dataset_source="Snowflake",
  name="SklearnMulitClassModel",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

#### Binary Classfication

In [ ]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_breast_cancer() 
feature_names = m_dataset.feature_names
u_features = [feature.replace(" ","_") for feature in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
session

In [ ]:
from fosforml import register_model
register_model(
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="Binary_Model",
  dataset_source = "SNOWFLAKE_STAGE",
  name="SklearnBinaryModel",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

#### Regression

In [ ]:
from sklearn.ensemble._forest import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_diabetes() 
feature_names = m_dataset.feature_names
u_features = [feature.replace(" ","_") for feature in feature_names]

m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target

x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestRegressor()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

In [ ]:
from fosforml import register_model
register_model(
  name="SklearnReyhtyjhgressionModel",
  model_obj=model,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RegressionModel",
  dataset_source = "SNOWFLAKE_STAGE",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"]
)